# Dynamic Programming

In [1]:
import numpy as np

"""
Given a primitive calculator that can perform the following three operations \
with the current number x: multiply x by 2, multiply x by 3, or add 1 to x. \
Your goal is given a positive integer n, find the minimum number of operations \
needed to obtain the number n starting from the number 1.
Args:
    target: The number needs to be splitted
Returns:
    The minimum number of operations and the calculation process.
"""
def PrimSplit(target):
    if target <= 0:
        raise ValueError("Input must be positive integers.")
    elif target == 1:
        return 0, [1]
    
    if target%3 == 0:
        operation3 = PrimSplit(target//3)
    else:
        operation3 = (np.nan, None)
        
    if target%2 == 0:
        operation2 = PrimSplit(target//2)
    else:
        operation2 = (np.nan, None)
    
    operation1 = PrimSplit(target-1)
    
    min_operation = np.nanmin([operation1[0], operation2[0], operation3[0]])
    if operation3[0] == min_operation:
        operation_num = operation3[0]+1 
        operation_process = operation3[1]
    elif operation2[0] == min_operation:
        operation_num = operation2[0]+1 
        operation_process = operation2[1]        
    else:
        operation_num = operation1[0]+1 
        operation_process = operation1[1]
        
    operation_process.append(target)
    return operation_num, operation_process

PrimSplit(11)

(4, [1, 3, 9, 10, 11])

In [2]:
"""
Find the best plan to take the gold with the pack number and weight restrictions.
Args:
    restrictions: The first element is weight, the second is pack number
    gold: The list of the gold with different weight
Returns:
    The best value we can get given the restrictions
"""
def TakeGold(restrictions, gold_list):
    pack_limit = restrictions[0]
    weight_limit = restrictions[1]
    
    if (pack_limit <= 0) or (len(gold_list) == 0) or (weight_limit-gold_list[0] < 0):
        return 0
    else:
        gold_list_new = gold_list.copy()
        new = gold_list_new.pop(0)
        plan_exclude = TakeGold([pack_limit, weight_limit], gold_list_new)
        plan_include = TakeGold([pack_limit-1, weight_limit-new], gold_list_new) + new
        max_value = max(plan_exclude, plan_include)
        
    return max_value


def BestValue(restrictions, gold):
    gold_list = sorted(gold)
    return TakeGold(restrictions, gold_list)

BestValue([3, 10], [1,4,8])

9

In [11]:
# Editing Distance Naive Method
# Start the recursion from the last element, and many subgroups are calculated repeatedly
# The worst complicity can be O(2^n)
"""
Calculate the edit distance between two strings (fewest steps to make two strings same \
by removing, inserting and replacing)
Args:
    string1: The first string
    string2: The second string
    m: The index of first string, used to work on the subset
    n: The index of the second string, used to work on the subset
Returns:
    The fewest steps to make those two strings same
"""
def EditDistance_naive(string1, string2, m=None, n=None):
    if m == None:
        m = len(string1)
    if n == None:
        n = len(string2)
    
    if m == 0:
        return n
    if n == 0:
        return m
    
    insert_step = EditDistance_naive(string1, string2, m-1, n) + 1
    remove_step = EditDistance_naive(string1, string2, m, n-1) + 1
    
    if string1[m-1] == string2[n-1]:
        replace_step = EditDistance_naive(string1, string2, m-1, n-1)
    else:
        replace_step = EditDistance_naive(string1, string2, m-1, n-1) + 1
    
    return min(insert_step, remove_step, replace_step)

EditDistance_naive("editing", "distance")

5

In [12]:
# Editing Distance Matrix Method 
# Start the loop from the first element, results of subgroups are stored so they won't be calculated repeatedly
# The complicity is O(len1*len2)
"""
Calculate the edit distance between two strings (fewest steps to make two strings same \
by removing, inserting and replacing)
Args:
    string1: The first string
    string2: The second string
Returns:
    The fewest steps to make those two strings same
"""
def EditDistance_matrix(string1, string2):
    m = len(string1)
    n = len(string2)
    
    result = [[None]*(n+1) for i in range(m+1)]
    
    for i in range(m+1):
        for j in range(n+1):
            if i == 0:
                result[i][j] = j
            elif j == 0:
                result[i][j] = i
            elif string1[i-1] == string2[j-1]:
                result[i][j] = result[i-1][j-1]
            else:
                result[i][j] = 1 + min(result[i-1][j],
                                       result[i][j-1],
                                       result[i-1][j-1])
    return result[m][n]    

EditDistance_matrix("editing", "distance")

5

In [25]:
# Maximum an arithmetic expression
"""
Operation Fun
"""
def calc(a, b, op):
    if op == "+":
        return a + b
    elif op == "-":
        return a - b
    elif op == "*":
        return a * b
    else:
        assert False

"""
Find the minimum and the maximum value for specific area in matrix
Args:
    i: Starting point
    j: Ending point
    op: Operation list
    m: Minimum value matrix
    M: Maximum value matrix
Returns:
    The maximum value and the minimum value
"""
def MinMax(i, j, op, m, M):
    mmin = float("Inf")
    Mmax = float("-Inf")
    for k in range(i, j):
        a = calc(m[i][k], m[k+1][j], op[k])
        b = calc(m[i][k], M[k+1][j], op[k])
        c = calc(M[i][k], m[k+1][j], op[k])
        d = calc(M[i][k], M[k+1][j], op[k])
        mmin = min(mmin, a, b, c, d)
        Mmax = max(Mmax, a, b, c, d)
    return mmin, Mmax

"""
Maximum the value of an arithmetic expression
Args:
    expression: The expression to be splitted
Returns:
    The maximum value of the expression 
"""
def MaxExpr(expression):
    digit = [int(expression[i]) for i in range(0,len(expression),2)]
    op = expression[1:len(expression):2]
    
    n = len(digit)
    m = [[None]*n for i in range(n)]
    M = [[None]*n for i in range(n)]
    
    # when length of parenthesis is 0, value is itself
    for i in range(n):
        m[i][i] = digit[i]
        M[i][i] = digit[i]
    
    # loop for each length>0
    for l in range(1, n):
        for i in range(n-l):
            j = i + l
            m[i][j], M[i][j] = MinMax(i, j, op, m, M)
    return M[0][n-1]

MaxExpr("5-8+7*4-8+9")

200

In [17]:
# Common substring of two strings
"""
Calculate the length of common substring of two strings
Args:
    string1: The first string
    string2: The second string
Returns:
    Length of the common substring
"""
def SubString(string1, string2):
    m = len(string1)
    n = len(string2)
    
    result = [[None]*(n+1) for i in range(m+1)]
    
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0:
                result[i][j] = 0
            elif string1[i-1] == string2[j-1]:
                result[i][j] = result[i-1][j-1] + 1
            else:
                result[i][j] = max(result[i-1][j],
                                   result[i][j-1])
    return result[m][n]

SubString("AGGTAB", "GXTXAYB")

4

In [37]:
# Common substring of three strings
"""
Calculate the length of common substring of three strings
Args:
    string1: The first string
    string2: The second string
    string3: The third string
Returns:
    Length of the common substring
"""
def SubString(string1, string2, string3):
    m = len(string1)
    n = len(string2)
    l = len(string3)
    
    result = [[[None]*(l+1) for i in range(n+1)] for j in range(m+1)]
    
    for i in range(m+1):
        for j in range(n+1):
            for k in range(l+1):
                if i == 0 or j == 0 or k == 0:
                    result[i][j][k] = 0
                elif string1[i-1] == string2[j-1] == string3[k-1]:
                    result[i][j][k] = result[i-1][j-1][k-1] + 1
                else:
                    result[i][j][k] = max(result[i-1][j][k],
                                       result[i][j-1][k],
                                       result[i][j][k-1])
    return result[m][n][l]

SubString("83217", "8213897", "683147")

3